In [ ]:
!pip install pandas scikit-learn nltk


In [ ]:

!mkdir -p ~/.kaggle
!cp /kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list


ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                        Student Performance Factors                          94KB  2024-09-02 10:53:57          23440        424  1.0              
abdulszz/spotify-most-streamed-songs                           Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14           8235        104  1.0              
ayushi10kumari/top-rated-movie-dataset                         Top Rated Movie Dataset                               1MB  2024-09-07 17:38:12           2040         23  0.9411765        
valakhorasani/electric-vehicle-charging-patterns               El

In [ ]:

!kaggle datasets download -d pypiahmad/goodreads-book-reviews
!unzip goodreads-book-reviews.zip


Dataset URL: https://www.kaggle.com/datasets/pypiahmad/goodreads-book-reviews
License(s): apache-2.0
goodreads-book-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  goodreads-book-reviews.zip
replace goodreads_reviews_dedup.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
df = pd.read_json('goodreads_reviews_spoiler.json', lines = True)
df.head()
df.columns

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378033 entries, 0 to 1378032
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   user_id           1378033 non-null  object        
 1   timestamp         1378033 non-null  datetime64[ns]
 2   review_sentences  1378033 non-null  object        
 3   rating            1378033 non-null  int64         
 4   has_spoiler       1378033 non-null  bool          
 5   book_id           1378033 non-null  int64         
 6   review_id         1378033 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(2), object(3)
memory usage: 64.4+ MB


In [ ]:
df['has_spoiler'].value_counts()

,count
has_spoiler,
False,1288406
True,89627


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        if isinstance(sentence, str):
            sentence = re.sub(r'[^\w\s]', '', sentence)
            sentence = sentence.lower()
            sentence = ' '.join([word for word in sentence.split() if word not in stopwords.words('english')])
        cleaned_sentences.append(sentence)
    return cleaned_sentences

df['cleaned_text'] = df['review_sentences'].apply(clean_text)
df['cleaned_text'].fillna('', inplace=True)

X_train = df['cleaned_text'].astype(str).tolist()
X_test = df['cleaned_text'].astype(str).tolist()

# Recheck for empty documents again
empty_docs_train = [doc for doc in X_train if len(doc.strip()) == 0]
empty_docs_test = [doc for doc in X_test if len(doc.strip()) == 0]

print(f"Number of empty documents in training set after cleaning: {len(empty_docs_train)}")
print(f"Number of empty documents in testing set after cleaning: {len(empty_docs_test)}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-33-c584072d1c11>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cleaned_text'].fillna('', inplace=True)


Number of empty documents in training set after cleaning: 0
Number of empty documents in testing set after cleaning: 0


In [ ]:
from sklearn.model_selection import train_test_split
X = df['cleaned_text']
Y = df['has_spoiler']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"Training data size: {len(X_train)}")
print(f"Testing data size: {len(X_test)}")

Training data size: 1102426
Testing data size: 275607


In [ ]:
empty_docs_train = [doc for doc in X_train if len(doc.strip()) == 0]
empty_docs_test = [doc for doc in X_test if len(doc.strip()) == 0]

print(f"Number of empty documents in training set: {len(empty_docs_train)}")
print(f"Number of empty documents in testing set: {len(empty_docs_test)}")


Number of empty documents in training set: 0
Number of empty documents in testing set: 0


In [ ]:
sample_size = 10
sample_data = df[['review_sentences', 'cleaned_text']].sample(n=sample_size, random_state=42)

print(sample_data)


                                          review_sentences  \
147051   [[0, Not sure I'm going to write a full review...   
1322675  [[0, Hooray this book is finally out!], [0, An...   
1221889  [[0, I think I'm losing interest in NA.], [0, ...   
1008531  [[0, This book features Michael, the man who d...   
859048   [[0, As with the first book in the Kitty Norvi...   
399220   [[0, There were a few grammar and sentence lev...   
799969   [[0, Amazing urban fantasy.], [0, For once mag...   
993745   [[0, Think this would have been perfect just h...   
1188494  [[0, I liked Angel Fire more than Angel - whic...   
1305916  [[0, Overall a good read!], [0, A little slow ...   

                                              cleaned_text  
147051   [[0, Not sure I'm going to write a full review...  
1322675  [[0, Hooray this book is finally out!], [0, An...  
1221889  [[0, I think I'm losing interest in NA.], [0, ...  
1008531  [[0, This book features Michael, the man who d...  
859048   [[0

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=10000)

# Fit the vectorizer on the training data and transform both training and testing data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Training TF-IDF matrix shape: {X_train_tfidf.shape}")
print(f"Testing TF-IDF matrix shape: {X_test_tfidf.shape}")


Training TF-IDF matrix shape: (1378033, 10000)
Testing TF-IDF matrix shape: (1378033, 10000)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'df' is your original DataFrame with 'cleaned_text' and 'spoiler' columns.

# Filter out rows where the cleaned_text is empty
df_filtered = df[df['cleaned_text'].str.strip() != '']

# Reset index for consistency
df_filtered.reset_index(drop=True, inplace=True)

# Prepare features and labels
X = df_filtered['cleaned_text']  # Text data
y = df_filtered['has_spoiler']  # Spoiler labels

# Now split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to list of strings if they are not already
X_train = X_train.astype(str).tolist()
X_test = X_test.astype(str).tolist()

# Transform the training and testing data into TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Check the shapes again
print(f"Shape of X_train_tfidf: {X_train_tfidf.shape}")  # Should match y_train
print(f"Shape of y_train: {len(Y_train)}")  # Should now match X_train_tfidf


Shape of X_train_tfidf: (1102426, 10000)
Shape of y_train: 1102426


In [ ]:

print(f"Shape of X_train_tfidf: {X_train_tfidf.shape}")  # Features
print(f"Shape of Y_train: {len(Y_train)}")  # Labels


Shape of X_train_tfidf: (1378033, 10000)
Shape of Y_train: 1102426


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=100, verbose=1)
model.fit(X_train_tfidf, Y_train)

X_test_tfidf = vectorizer.transform(X_test)
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(Y_test, y_pred)
report = classification_report(Y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.9356801532617096
Classification Report:
              precision    recall  f1-score   support

       False       0.94      0.99      0.97    257570
        True       0.55      0.09      0.16     18037

    accuracy                           0.94    275607
   macro avg       0.75      0.54      0.56    275607
weighted avg       0.91      0.94      0.91    275607

